In [6]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import models, transforms


In [7]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

resnet = models.resnet50(pretrained=True)
resnet = resnet.to(device)
resnet.eval()

# We will use the LAST convolutional layer
target_layer = resnet.layer4[-1]

print("✓ ResNet50 loaded for Grad-CAM")


✓ ResNet50 loaded for Grad-CAM


In [8]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [9]:
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None

        self._register_hooks()

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output.detach()

        def backward_hook(module, grad_in, grad_out):
            self.gradients = grad_out[0].detach()

        self.target_layer.register_forward_hook(forward_hook)
        self.target_layer.register_backward_hook(backward_hook)

    def generate(self, input_tensor, class_idx=None):
        self.model.zero_grad()

        output = self.model(input_tensor)

        # Since it's regression-like usage, take mean activation
        score = output.mean()
        score.backward()

        weights = self.gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1)

        cam = F.relu(cam)
        cam = cam.squeeze().cpu().numpy()

        cam = cv2.resize(cam, (224, 224))
        cam = (cam - cam.min()) / (cam.max() + 1e-8)

        return cam


In [10]:
# Manually select representative property IDs
sample_property_ids = [
    7129300520,  # example high-value
    6414100192,  # urban
    1925069082,  # suburban
    3655000070,  # greenery
    1180000207,  # dense area
    8562750320   # random
]

image_dir = 'data/images/train'


In [17]:
def find_image_path(image_dir, prop_id):
    for ext in ['.png', '.jpg', '.jpeg']:
        path = os.path.join(image_dir, f"{prop_id}{ext}")
        if os.path.exists(path):
            return path
    return None


image_dir = '../data/images/train'
os.makedirs('../outputs/cam_visualizations', exist_ok=True)

available_ids = [
    int(os.path.splitext(f)[0])
    for f in os.listdir(image_dir)
    if f.lower().endswith(('.png', '.jpg', '.jpeg'))
]

sample_property_ids = available_ids[:6]

gradcam = GradCAM(resnet, target_layer)

for prop_id in sample_property_ids:
    img_path = find_image_path(image_dir, prop_id)

    if img_path is None:
        continue

    img = Image.open(img_path).convert("RGB")
    img_tensor = image_transform(img).unsqueeze(0).to(device)

    cam = gradcam.generate(img_tensor)

    img_np = np.array(img.resize((224, 224)))
    heatmap = cv2.applyColorMap(
        np.uint8(255 * cam),
        cv2.COLORMAP_JET
    )

    overlay = cv2.addWeighted(img_np, 0.6, heatmap, 0.4, 0)

    out_path = f'../outputs/cam_visualizations/{prop_id}_gradcam.png'
    cv2.imwrite(out_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))

    print(f"✓ Saved Grad-CAM for property {prop_id}")


/Users/macbookpro/Documents/Cursor Projects/CDC /satellite-property-valuation/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1866: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


✓ Saved Grad-CAM for property 3885802970
✓ Saved Grad-CAM for property 5318101040
✓ Saved Grad-CAM for property 7885800160
✓ Saved Grad-CAM for property 5608000860
✓ Saved Grad-CAM for property 7169500020
✓ Saved Grad-CAM for property 1442300035
